Data Loading and basic data exploration

In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [3]:
df = pd.read_csv("all_regions_combined_data.csv")

## Basic Info  of the data 
print("Dataset shape",df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nData types:")
print(df.dtypes)
print("\nMissing Values")
print(df.isnull().sum())
print("\nUnique values per column:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()}")
    

Dataset shape (175320, 11)

Column names:
['Datetime (UTC)', 'Country', 'Zone name', 'Zone id', 'Carbon intensity gCO₂eq/kWh (direct)', 'Carbon intensity gCO₂eq/kWh (Life cycle)', 'Carbon-free energy percentage (CFE%)', 'Renewable energy percentage (RE%)', 'Data source', 'Data estimated', 'Data estimation method']

Data types:
Datetime (UTC)                               object
Country                                      object
Zone name                                    object
Zone id                                      object
Carbon intensity gCO₂eq/kWh (direct)        float64
Carbon intensity gCO₂eq/kWh (Life cycle)    float64
Carbon-free energy percentage (CFE%)        float64
Renewable energy percentage (RE%)           float64
Data source                                  object
Data estimated                                 bool
Data estimation method                       object
dtype: object

Missing Values
Datetime (UTC)                              0
Country                

Data Cleaning 

In [4]:
# Convert datetime column
df['Datetime (UTC)'] = pd.to_datetime(df['Datetime (UTC)'])
# Clean column names (remove special characters and spaces)
df.columns = df.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('/', '_')
# Check for and handle missing values
print("Missing values after cleaning:")
print(df.isnull().sum())

# All columns with numerical values 
numeric_cols = ['Carbon_intensity_gCO₂eq_kWh_direct', 'Carbon_intensity_gCO₂eq_kWh_Life_cycle', 
                'Carbon-free_energy_percentage_CFE%', 'Renewable_energy_percentage_RE%']
# Checking fo rouliers using Inter Quartile range 
def detect_outliers(df,column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1 
    low_bund = Q1 - 1.5 *IQR
    upper_bund = Q1 + 1.5 *IQR
    outliers = df[(df[column]< low_bund)|(df[column]>upper_bund)]
    return outliers,low_bund,upper_bund

# To check outliers for each  numerical column 
for col in numeric_cols :
    outliers,lower,upper = detect_outliers(df,col)
    print(f"\n{col}") 
    print(f"Outliers: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")
    print(f"Range: {lower:.2f} to {upper:.2f}")

Missing values after cleaning:
Datetime_UTC                              0
Country                                   0
Zone_name                                 0
Zone_id                                   0
Carbon_intensity_gCO₂eq_kWh_direct        0
Carbon_intensity_gCO₂eq_kWh_Life_cycle    0
Carbon-free_energy_percentage_CFE%        0
Renewable_energy_percentage_RE%           0
Data_source                               0
Data_estimated                            0
Data_estimation_method                    0
dtype: int64

Carbon_intensity_gCO₂eq_kWh_direct
Outliers: 1 (0.00%)
Range: 91.40 to 770.62

Carbon_intensity_gCO₂eq_kWh_Life_cycle
Outliers: 1 (0.00%)
Range: 150.77 to 825.62

Carbon-free_energy_percentage_CFE%
Outliers: 5078 (2.90%)
Range: -24.18 to 49.46

Renewable_energy_percentage_RE%
Outliers: 8549 (4.88%)
Range: -23.36 to 45.22


Feature Engineering 

In [5]:
df['Year'] = df['Datetime_UTC'].dt.year
df['Month'] = df['Datetime_UTC'].dt.month
df['Day'] = df['Datetime_UTC'].dt.day
df['Hour'] = df['Datetime_UTC'].dt.hour
df['DayOfWeek'] = df['Datetime_UTC'].dt.dayofweek
df['WeekOfYear'] = df['Datetime_UTC'].dt.isocalendar().week
df['Quarter'] = df['Datetime_UTC'].dt.quarter 

#Create time-based categories
df['Season'] = df['Month'].map({12: 'Winter', 1: 'Winter', 2: 'Winter',
                                3: 'Spring', 4: 'Spring', 5: 'Spring',
                                6: 'Summer', 7: 'Summer', 8: 'Summer',
                                9: 'Autumn', 10: 'Autumn', 11: 'Autumn'})

df['TimeOfDay'] = pd.cut(df['Hour'], 
                        bins=[0, 6, 12, 18, 24], 
                        labels=['Night', 'Morning', 'Afternoon', 'Evening'])

# Create efficiency metrics
df['Carbon_Efficiency'] = df['Renewable_energy_percentage_RE%'] / df['Carbon_intensity_gCO₂eq_kWh_direct']
df['Clean_Energy_Gap'] = df['Carbon-free_energy_percentage_CFE%'] - df['Renewable_energy_percentage_RE%']

# Regional encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Zone_encoded'] = le.fit_transform(df['Zone_name'])

# Create moving averages for trend analysis
df = df.sort_values(['Zone_name', 'Datetime_UTC'])
df['Carbon_Intensity_MA_24h'] = df.groupby('Zone_name')['Carbon_intensity_gCO₂eq_kWh_direct'].rolling(window=24, min_periods=1).mean().reset_index(0, drop=True)
df['RE_Percentage_MA_24h'] = df.groupby('Zone_name')['Renewable_energy_percentage_RE%'].rolling(window=24, min_periods=1).mean().reset_index(0, drop=True)

Data Normalization

In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Identify numerical columns for scaling
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Remove ID-like columns
numerical_cols = [col for col in numerical_cols if col not in ['Year', 'Month', 'Day', 'Hour', 'Zone_encoded']]

# Option 1: StandardScaler (mean=0, std=1)
scaler_standard = StandardScaler()
df_scaled_standard = df.copy()
df_scaled_standard[numerical_cols] = scaler_standard.fit_transform(df[numerical_cols])

# Option 2: MinMaxScaler (range 0-1)
scaler_minmax = MinMaxScaler()
df_scaled_minmax = df.copy()
df_scaled_minmax[numerical_cols] = scaler_minmax.fit_transform(df[numerical_cols])

# Save scalers for future use
import joblib
joblib.dump(scaler_standard, 'standard_scaler.pkl')
joblib.dump(scaler_minmax, 'minmax_scaler.pkl')

['minmax_scaler.pkl']

In [7]:
# Save the preprocessed data to CSV file
# Use the standard scaled version (or df_scaled_minmax if you prefer MinMax scaling)
df_scaled_standard.to_csv('processed_data.csv', index=False)

print("Preprocessed data saved as 'processed_data.csv'")
print(f"Shape: {df_scaled_standard.shape}")
print(f"Columns: {len(df_scaled_standard.columns)}")

# Also save some metadata about the preprocessing
preprocessing_info = {
    'original_shape': df.shape,
    'processed_shape': df_scaled_standard.shape,
    'numerical_columns_scaled': numerical_cols,
    'scaler_used': 'StandardScaler',
    'target_column': 'Carbon_intensity_gCO₂eq_kWh_direct',
    'datetime_column': 'Datetime_UTC'
}

import json
with open('preprocessing_info.json', 'w') as f:
    json.dump(preprocessing_info, f, indent=2, default=str)

print("Preprocessing info saved as 'preprocessing_info.json'")

Preprocessed data saved as 'processed_data.csv'
Shape: (175320, 25)
Columns: 25
Preprocessing info saved as 'preprocessing_info.json'
